#GDA Project Group 2

##Profiling hate speach spreders on Twitter


1. Understand the content that is available in the dataset .
2. Pre-process the data to improve the final results.
3. SVM model
a. Follow a Support Vector Machine (SVM) based approach to classify users.
b. Use tfidf as input for SVMs
c. Analyze the outcomes.

OPTIONAL:
d. Test different lengths of N-Grams and report the best N in which the classifier
achieved the best performance 
e. Report the top 10 representative (most repeated) 1, 2 and 3-grams for each of
the classes.

4. LSTM model
a. Build a LSTM (Long Short-term Memory) text classifier with the same goal as
above.
b. Analyze the outcomes.
6. Compare the results of both models using the F1 measure.
7. Obtain statistics that can corroborate the results achieved and explain the reason for
these choices.



**Additional Hints**

You can use whatever data you can find additionally to the data we provided, but
don’t spend too much time on searching.
This project is about applying the skills you learned during GDA. It is not about
developing the perfect solution for a given problem.
The project might be a little bit harder than what you are used to. Take it as a “bridge”
to the practice phase.
Your trainers are available in case you need a nudge in the right direction.


In [1]:
# Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import xml.etree.ElementTree as ET
enlist=[]
#Loop over all files in current directory (change directory if necessary)
for file in os.listdir(os.getcwd()):
    #get current file
    current_file = os.path.join(os.getcwd(), file)
    #process all files having 'xml' in name
    if 'xml' in current_file:
        data = open(current_file, "r")
        #Parse XML tree
        tree = ET.parse(data)
        root = tree.getroot()
        #Get class attribute
        cls=root.attrib['class']
        #Find all documents in file
        docs=root.find('documents')
        #Append documents to the list 'texts'
        texts = [doc.text for doc in docs]
        #Create a dictionary containing file names, classes and texts
        enlist.append({'file':file,'class':cls,'texts':texts})
import pandas as pd
endf=pd.DataFrame(enlist)
len(endf)
            

NameError: name 'os' is not defined

In [49]:
pip install pandas_read_xml

In [50]:
# More at:
# https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.read_xml.html
# https://stackoverflow.com/questions/40554657/module-pandas-has-no-attribute-read-csv/40554730

# This is just for understand how this read xml works.
import pandas as pd
import pandas_read_xml as pdx
from pandas_read_xml import flatten

# As you can see, loaded Data Frame has only one ROW. It needs further processing.
df = pdx.read_xml("/content/drive/My Drive/Colab Notebooks/GDA Project Group 2 /pan21-author-profiling-training-2021-03-14/en/0a3ce42bea89e2a92a28f685735e605e.xml")
df.head()

,author
0,"{'@lang': 'en', '@class': '1', 'documents': {'..."


In [51]:
# Data Frame needs to be 3 times flatten to reach expected shape.
df = df.pipe(flatten).pipe(flatten).pipe(flatten)
df.head()

,author|@lang,author|@class,author|documents|document
0,en,1,"""Hey Jamal (snickering uncontrollable) You wan..."
1,en,1,RT #USER#: Cotton coming out with a banger #URL#
2,en,1,This is meant to be sarcasm but it's a good po...
3,en,1,Nick really just compared homosexuality to peo...
4,en,1,PROTECT AMERICA FIRST! LET'S GO!!!!!!!! #URL#


In [52]:
df['author|documents|document'].iloc[0]

'"Hey Jamal (snickering uncontrollable) You want some (PFFF) LEMONADE!" What an IDIOT! #URL#'

In [53]:
my_file1 = open("/content/drive/My Drive/Colab Notebooks/GDA Project Group 2 /pan21-author-profiling-training-2021-03-14/en/truth.txt", "r")
content = my_file1.read()
content_list_en = content.split("\n")
for i in content_list_en:
  print(i.split(':'))

['639b8e5e6a527d494c85d8f5704b1a01', '', '', '0']
['f2b1fc84c500c38a93522efbd422b559', '', '', '0']
['10b2d013382e1fb3c9414ea28329f258', '', '', '0']
['26644d1348fc1122e8c5ef45d6bc84fa', '', '', '0']
['4d4c5dcbfe38d0d33a0d1b1419952ca8', '', '', '0']
['86a7f84c2dd126dac46270b6c912952c', '', '', '0']
['88d8ac4ae2d91e681ee8a8d3b60fd911', '', '', '0']
['2a9776a2909ba5e27043faf2d57ba410', '', '', '0']
['fdb47a3f65091b9a5b989e1722c9fac4', '', '', '0']
['5625ca3bc44c89d763221b663f7c0987', '', '', '0']
['9d08913250938aadcf6c18d0c89a0d14', '', '', '0']
['5027d4e88fc5179e027a12872dfe6835', '', '', '0']
['e30577895cbb4d8f073e4dbb2f3be6d1', '', '', '0']
['06893abba0bb8f94fed7562350233ed7', '', '', '0']
['6fd5e2e31d6c2b2eb4b5dc07280636a7', '', '', '0']
['8b525999b04b19255b32365e49b281ac', '', '', '0']
['58584745632b5367da1c7a9af746222b', '', '', '0']
['9d58d6313bfb2fba9e1e45bb9d65cf0b', '', '', '0']
['e6e216a24993a72629629da7c182c3af', '', '', '0']
['94b5cb02328011bbecaf78d79d2c1ff5', '', '', '0']


In [54]:
# Try to load EN files
from tqdm import tqdm # Wery nice feature indeed :)
df_en  = pd.DataFrame(columns=['author|@lang',	'author|@class',	'author|documents|document'])
for i in tqdm(content_list_en):
  ID = i.split(':')[0] # Take ID from current line
  Mark = i.split(':')[-1] # Tke Mark from current line
  # Load and process current XML file
  df1 = pdx.read_xml(f"/content/drive/My Drive/Colab Notebooks/GDA Project Group 2 /pan21-author-profiling-training-2021-03-14/en/{ID}.xml")
  df1 = df.pipe(flatten).pipe(flatten).pipe(flatten)
  # Add identification collumns to dataframe
  df1['ID'] = ID
  df1['Mark'] = Mark
  # add temporarry DF to Final
  df_en = pd.concat([df_en , df1], ignore_index=True)
df_en

100%|██████████| 200/200 [00:05<00:00, 39.37it/s]


,author|@lang,author|@class,author|documents|document,ID,Mark
0,en,0,RT #USER#: Funny how “15 days to slow the spre...,639b8e5e6a527d494c85d8f5704b1a01,0
1,en,0,RT #USER#: Why did Minneapolis just give Georg...,639b8e5e6a527d494c85d8f5704b1a01,0
2,en,0,"RT #USER#: To be fair, he has done a lot of un...",639b8e5e6a527d494c85d8f5704b1a01,0
3,en,0,RT #USER#: President Trump got us the #HASHTAG...,639b8e5e6a527d494c85d8f5704b1a01,0
4,en,0,RT #USER#: Is the case against former officer ...,639b8e5e6a527d494c85d8f5704b1a01,0
...,...,...,...,...,...
39995,en,1,You got to fuck her like you miss her everytim...,41501686277ace6b5fd7dcfe9284fe1d,1
39996,en,1,i'm solid. been solid. will forever be solid. ...,41501686277ace6b5fd7dcfe9284fe1d,1
39997,en,1,You nap town bitches really stepped on .... ta...,41501686277ace6b5fd7dcfe9284fe1d,1
39998,en,1,REPEAT AFTER ME : I don't owe nobody shit !,41501686277ace6b5fd7dcfe9284fe1d,1


In [64]:
# Try to load ES files

my_file2 = open("/content/drive/My Drive/Colab Notebooks/GDA Project Group 2 /pan21-author-profiling-training-2021-03-14/es/truth.txt", "r")
content = my_file2.read()
content_list_es = content.split("\n")

from tqdm import tqdm # Wery nice feature indeed :)
df_es  = pd.DataFrame(columns=['author|@lang',	'author|@class',	'author|documents|document'])
for i in tqdm(content_list_es):
  ID = i.split(':')[0] # Take ID from current line
  Mark = i.split(':')[-1] # Tke Mark from current line
  # Load and process current XML file
  df2 = pdx.read_xml(f"/content/drive/My Drive/Colab Notebooks/GDA Project Group 2 /pan21-author-profiling-training-2021-03-14/es/{ID}.xml")
  df2 = df.pipe(flatten).pipe(flatten).pipe(flatten)
  # Add identification collumns to dataframe
  df2['ID'] = ID
  df2['Mark'] = Mark
  # add temporarry DF to Final
  df_es = pd.concat([df_en , df2], ignore_index=True)
  print(ID,Mark)
  break
# df_es

  0%|          | 0/200 [00:00<?, ?it/s]

1a698d93874644625d83a76b206c7aa3 0


In [ ]:
df_es